In [1]:
%%script C:\Users\Jan Catherine\AppData\Local\Programs\Python\Python38\python.exe
import sys
print(sys.version)

3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [2]:
pwd

'C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN'

In [3]:
import os
os.chdir('../../Data_Collection_02/02_Preprocessing/05_CNN')

In [4]:
#pip install tensorflow

In [5]:
#importing libraries
"""
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
"""

'\nfrom keras.models import Sequential\nfrom keras.layers import Dense, Dropout, Activation, Flatten\nfrom keras.layers import Convolution2D, MaxPooling2D\nfrom sklearn.model_selection import train_test_split\nimport pandas as pd\nimport numpy as np\n'

In [6]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy


from sklearn.model_selection import KFold

#import pydot

In [7]:
import re
dataset = pd.read_csv('01_URL_Train.csv', encoding='latin-1')
testdata = pd.read_csv('01_URL_Test.csv', encoding='latin-1')

train_data = dataset
test_data = testdata

#Oversampling
count_class_0, count_class_1 = train_data.Fake.value_counts()

df_class_0 = train_data[train_data['Fake'] == 0]
df_class_1 = train_data[train_data['Fake'] == 1]

df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

df_test_over = df_test_over.reset_index(drop=True)
print('Random over-sampling:')
print(df_test_over.Fake.value_counts())

train_data = df_test_over

print(train_data.Fake.value_counts())

train_data.dropna(axis=0, how ='any', inplace=True)
train_data['Num_words_url'] = train_data['URL'].apply(lambda x:len(re.findall(r"[\w']+", str(x))))

print('-------Train data--------')
print(train_data['Fake'].value_counts())
print(len(train_data))
print('-------------------------')

max_train_url_length  = train_data['Num_words_url'].max()



test_data.dropna(axis = 0, how ='any',inplace=True) 
test_data['Num_words_url'] = test_data['URL'].apply(lambda x:len(re.findall(r"[\w']+", str(x))))

max_test_url_length  = test_data['Num_words_url'].max()

print('-------Test data--------')
print(test_data['Fake'].value_counts())
print(len(test_data))
print('-------------------------')

print('Train Max Length :'+str(max_train_url_length))
print('Test Max Sentence Length :'+str(max_test_url_length))


Random over-sampling:
0    5760
1    5760
Name: Fake, dtype: int64
0    5760
1    5760
Name: Fake, dtype: int64
-------Train data--------
0    5760
1    5760
Name: Fake, dtype: int64
11520
-------------------------
-------Test data--------
0    1429
1     184
Name: Fake, dtype: int64
1613
-------------------------
Train Max Length :37
Test Max Sentence Length :37


In [8]:
train_data.head()

,URL,Fake,Num_words_url
0,https://www.thehindu.com/opinion/lead/a-key-ar...,0,16
1,https://www.mckinsey.com/industries/financial-...,0,22
2,https://coronadotimes.com/news/2020/06/14/nail...,0,17
3,https://aricjournal.biomedcentral.com/articles...,0,11
4,https://www.idealista.com/en/news/property-for...,0,27


In [9]:
num_words = 12000


URLword = Tokenizer(num_words=num_words, oov_token="unk", filters='!"#$%&()*+,./:;<=>?@[\\]^`{|}~\t\n',)
URLword.fit_on_texts(train_data['URL'].tolist())

print(str(URLword.texts_to_sequences(['https://www.thehindu.com/opinion/lead/&bill-gates012/100??0'])))


[[2, 3, 547, 4, 44, 3384, 1, 1, 612]]


In [19]:
print(str(URLword.texts_to_sequences(['https://www.thehindu.com/opinion/lead/gates/news/covid'])))

[[2, 3, 547, 4, 44, 3384, 1, 6, 1455]]


In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data['URL'].tolist(),\
                                                      train_data['Fake'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = train_data['Fake'].tolist(),\
                                                      random_state=0)

print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(y_train)))
print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(y_valid)))

x_train = np.array( URLword.texts_to_sequences(X_train) )
x_valid = np.array( URLword.texts_to_sequences(X_valid) )
x_test  = np.array( URLword.texts_to_sequences(test_data['URL'].tolist()) )

x_train = pad_sequences(x_train, padding='post', maxlen=40)
x_valid = pad_sequences(x_valid, padding='post', maxlen=40)
x_test = pad_sequences(x_test, padding='post', maxlen=40)

print(x_train[0])

le = LabelEncoder()

train_labels = le.fit_transform(y_train)
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels, 2))
#print(train_labels)
valid_labels = le.transform(y_valid)
valid_labels = np.asarray( tf.keras.utils.to_categorical(valid_labels, 2))


test_labels = le.transform(test_data['Fake'].tolist())
test_labels = np.asarray(tf.keras.utils.to_categorical(test_labels, 2))
list(le.classes_)


train_ds = tf.data.Dataset.from_tensor_slices((x_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,test_labels))

Train data len:9216
Class distributionCounter({0: 4608, 1: 4608})
Valid data len:2304
Class distributionCounter({1: 1152, 0: 1152})


C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_9208\3143876583.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array( URLword.texts_to_sequences(X_train) )
C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_9208\3143876583.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_valid = np.array( URLword.texts_to_sequences(X_valid) )
C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_9208\3143876583.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or

[   2    3 1312    4    6  147 7678    8    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


In [11]:
test_data.head()

,URL,Fake,Num_words_url
0,https://bhekisisa.org/article/2020-06-03-coron...,0,25
1,https://www.express.co.uk/travel/articles/1246...,0,23
2,https://www.thesun.co.uk/tvandshowbiz/11851914...,0,12
3,https://www.inquirer.com/opinion/china-reparat...,0,15
4,https://scdhec.gov/covid19/covid-19-testing-lo...,0,8


In [12]:
count =0
print('======Train dataset ====')
for value,label in train_ds:
    count += 1
    print(value,label)
    if count==3:
        break
count =0
print('======Validation dataset ====')
for value,label in valid_ds:
    count += 1
    print(value,label)
    if count==3:
        break
count = 0
print('======Test dataset ====')
for value,label in test_ds:
    count += 1
    print(value,label)
    if count==3:
        break  


======Train dataset ====
tf.Tensor(
[   2    3 1312    4    6  147 7678    8    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0], shape=(40,), dtype=int32) tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor(
[   2 3927    4   22    5   14 3928    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0], shape=(40,), dtype=int32) tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor(
[   2    3 7563    9 1472 2773 7564    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0], shape=(40,), dtype=int32) tf.Tensor([1. 0.], shape=(2,), dtype=float32)
======Validation dataset ====
tf.Tensor(
[  2   3 521   4 522 523   0   0   0   0   0   0   0   0   0   0   0  

In [13]:
#CNN MODEL

max_features = 11718
embedding_dim = 64 #same as URLNet
sequence_length = 40

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

inputs = np.concatenate((x_train,x_valid), axis=0)
targets = np.concatenate((train_labels, valid_labels), axis=0)

#regularizer prevents overfitting
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):
    
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(max_features +1, embedding_dim, input_length=sequence_length,\
                                        embeddings_regularizer = regularizers.l2(0.0005)))                                    

    model.add(tf.keras.layers.Conv1D(128,3, activation='relu',\
                                     kernel_regularizer = regularizers.l2(0.0005),\
                                     bias_regularizer = regularizers.l2(0.0005)))                               


    model.add(tf.keras.layers.GlobalMaxPooling1D())

    model.add(tf.keras.layers.Dropout(0.5)) #to reduce overfitting

    #final classification, 2 classes
    model.add(tf.keras.layers.Dense(2, activation='sigmoid',\
                                    kernel_regularizer=regularizers.l2(0.001),\
                                    bias_regularizer=regularizers.l2(0.001),))




    model.summary()
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer='Nadam', metrics=["CategoricalAccuracy"])
    
    epochs = 100
    # Fit the model using the train and test datasets.
    #history = model.fit(x_train, train_labels,validation_data= (x_test,test_labels),epochs=epochs )
    history = model.fit(inputs[train], targets[train],
                        epochs= epochs ,
                        verbose=1)
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 64)            750016    
                                                                 
 conv1d (Conv1D)             (None, 38, 128)           24704     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 774978 (2.96 MB)
Trainable params: 774978 (2.96 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [14]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.1345098316669464 - Accuracy: 99.30555820465088%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.12136513739824295 - Accuracy: 99.47916865348816%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.137055903673172 - Accuracy: 99.13194179534912%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.12402170151472092 - Accuracy: 99.39236044883728%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.1365675926208496 - Accuracy: 99.21875%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 99.30555582046509 (+- 0.12276281154030338)
> Loss: 0.13070403337478637
----------------------------------

In [15]:
history.history

{'loss': [0.5313675403594971,
  0.19979873299598694,
  0.14098471403121948,
  0.12931157648563385,
  0.12532496452331543,
  0.12225338071584702,
  0.12276589870452881,
  0.12097842246294022,
  0.12222182750701904,
  0.11975150555372238,
  0.12034854292869568,
  0.12149621546268463,
  0.1195167824625969,
  0.11958762258291245,
  0.12122979760169983,
  0.12014549225568771,
  0.12011817842721939,
  0.11969167739152908,
  0.12030240148305893,
  0.12020524591207504,
  0.11931528151035309,
  0.1205085813999176,
  0.11900349706411362,
  0.12023723125457764,
  0.1190003827214241,
  0.12011093646287918,
  0.12037459760904312,
  0.1191348135471344,
  0.11945191770792007,
  0.11917661130428314,
  0.11931359767913818,
  0.11799805611371994,
  0.11785277724266052,
  0.11876413226127625,
  0.11834987252950668,
  0.11773988604545593,
  0.11944953352212906,
  0.11718656122684479,
  0.11892770230770111,
  0.11828948557376862,
  0.11819843202829361,
  0.11743625998497009,
  0.11842034757137299,
  0.1163

In [16]:
"""
plt.plot(history.history['loss'], label=' training data')
plt.plot(history.history['val_loss'], label='validation data')
plt.title('Loss for Text Classification')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()
"""

'\nplt.plot(history.history[\'loss\'], label=\' training data\')\nplt.plot(history.history[\'val_loss\'], label=\'validation data\')\nplt.title(\'Loss for Text Classification\')\nplt.ylabel(\'Loss value\')\nplt.xlabel(\'No. epoch\')\nplt.legend(loc="upper left")\nplt.show()\n'

In [17]:
"""
plt.plot(history.history['categorical_accuracy'], label=' (training data)')
plt.plot(history.history['val_categorical_accuracy'], label='CategoricalCrossentropy (validation data)')
plt.title('CategoricalAccuracy for Text Classification')
plt.ylabel('CategoricalAccuracy value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()
"""

'\nplt.plot(history.history[\'categorical_accuracy\'], label=\' (training data)\')\nplt.plot(history.history[\'val_categorical_accuracy\'], label=\'CategoricalCrossentropy (validation data)\')\nplt.title(\'CategoricalAccuracy for Text Classification\')\nplt.ylabel(\'CategoricalAccuracy value\')\nplt.xlabel(\'No. epoch\')\nplt.legend(loc="upper left")\nplt.show()\n'

In [40]:
for layer in model.layers:
    layer._name = layer.name+"_word"

In [41]:
model.save('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\URLword') 
json_string4 = URLword.to_json()

INFO:tensorflow:Assets written to: C:\Users\Jan Catherine\Documents\CMSC Notes\CMSC 190 Part 2\Codes\CNN\URLword\assets


INFO:tensorflow:Assets written to: C:\Users\Jan Catherine\Documents\CMSC Notes\CMSC 190 Part 2\Codes\CNN\URLword\assets


In [42]:
import json
with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\URLword\\URLword.json', 'w') as outfile4:
    json.dump(json_string4, outfile4)

In [31]:
#reload model

new_model = tf.keras.models.load_model('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\URLword')
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 40, 64)            750016    
                                                                 
 conv1d_4 (Conv1D)           (None, 38, 128)           24704     
                                                                 
 global_max_pooling1d_4 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 2)                 258       
                                                                 
Total params: 774978 (2.96 MB)
Trainable params: 774978 (2.96 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [32]:
with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\URLword\\URLword.json') as json_file:
    json_string = json.load(json_file)
tokenizer1 = tf.keras.preprocessing.text.tokenizer_from_json(json_string)

In [33]:
x_test  = np.array( tokenizer1.texts_to_sequences(test_data['URL'].tolist()) )
x_test = pad_sequences(x_test, padding='post', maxlen=40)

C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_9208\532273747.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test  = np.array( tokenizer1.texts_to_sequences(test_data['URL'].tolist()) )


In [34]:
# Generate predictions (probabilities -- the output of the last layer)
# on test  data using `predict`
print("Generate predictions for all samples")
predictions = new_model.predict(x_test)
print(predictions)
predict_results = predictions.argmax(axis=1)

Generate predictions for all samples
51/51 [==============================] - 0s 2ms/step
[[0.9864073  0.01359272]
 [0.9703854  0.02961461]
 [0.92938405 0.07061591]
 ...
 [0.9523344  0.0476656 ]
 [0.9263759  0.07362404]
 [0.9869116  0.01308842]]


In [35]:
test_data['pred_fake']= predict_results
test_data['pred_fake'] = np.where((test_data.pred_fake == '0'),0,test_data.pred_fake)
test_data['pred_fake'] = np.where((test_data.pred_fake == '1'),1,test_data.pred_fake)

In [36]:
print(type(test_data['Fake']))
test_data['Fake'].head()


<class 'pandas.core.series.Series'>


0    0
1    0
2    0
3    0
4    0
Name: Fake, dtype: int64

In [37]:
print(type(test_data['pred_fake']))
test_data['pred_fake'].head()

<class 'pandas.core.series.Series'>


0    0
1    0
2    0
3    0
4    0
Name: pred_fake, dtype: int64

In [38]:
labels = [0, 1]
    
print(classification_report(test_data['Fake'].tolist(),test_data['pred_fake'].tolist(),labels=labels))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      1429
           1       0.88      0.66      0.76       184

    accuracy                           0.95      1613
   macro avg       0.92      0.83      0.86      1613
weighted avg       0.95      0.95      0.95      1613



In [39]:
precisionscore = precision_score(y_test, new_model.predict(x_test))
accuracyscore = accuracy_score(y_test, new_model.predict(x_test))
recallscore = recall_score(y_test, new_model.predict(x_test))
f1score = f1_score(y_test, new_model.predict(x_test))
cm = confusion_matrix(y_test, new_model.predict(x_test))

print(precisionscore, accuracyscore, recallscore, f1score, cm)

NameError: name 'precision_score' is not defined

In [6]:
from tensorflow.python import keras
print(keras.__version__)

2.6.0


In [7]:
import tensorflow as tf
print(tf.__version__)

2.13.0
